In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.arduino import Arduino_IO

In [2]:
# Define an octave with naturals and sharps (Zz = rest)
Cn = 1
Cs = 2
Dn = 3
Ds = 4
En = 5
Fn = 6
Fs = 7
Gn = 8
Gs = 9
An = 10
As = 11
Bn = 12
Zz = 13

# Define another one with flats and remaining sharps
Bs = 1
Df = 2
Dn2 = 3
Ef = 4
En2 = 5
Es = 6
Gf = 7
Gn2 = 8
Af = 9
An2 = 10
Bf = 11
Bn2 = 12
Zz2 = 13

# PS - PL mode

In [ ]:
base = BaseOverlay("base.bit")

In [ ]:
arduino_pin_d8 = Arduino_IO(base.ARDUINO, 8, 'out')
arduino_pin_d9 = Arduino_IO(base.ARDUINO, 9, 'out')

In [ ]:
dirPin = arduino_pin_d8
stepPin = arduino_pin_d9

In [ ]:
# Frequencies in hundredths of Hz, e.g. middle A = 44000
# 4 Octaves with 12 notes per octave, i.e. C to B

freq = [[13081,13859,14683,15556,16481,17461,18500,19600,20765,22000,23308,24694],
[26163,27718,29366,31113,32963,34923,36999,39200,41530,44000,46616,49388],
[52325,55437,58733,62225,65925,69846,73999,78399,83061,88000,93233,98777],
[104650,110873,117466,124451,131851,139691,147998,156798,166122,176000,186466,197553]]

In [ ]:
# Frequency (in Hz) is converted to Floppy Delay using the formula:
#   314000 / frequency = floppy delay
# so middle A = 314000 / 440 = 714
#
# Lowest realistic note is delay = 1550
# Highest realistic note is delay = 210

floppyConv = 31400000
# Calculate all our floppy delays at the start
floppyDelay = [[0 for j in range(12)] for i in range(4)]
# Song1 is the C major scale (note, octave, length)
song1_tempo = 120

noteStatus = 0
actual_note = 0
actual_octave = 0

In [ ]:
def _resetMotor():
    # To reset head position move back 10 then forward 5
    dirPin.write(0)
    for i in range(10):
        stepPin.write(1)
        stepPin.write(0)
        time.sleep(0.001)

        dirPin.write(1)
    for i in range(5):
        stepPin.write(1)
        stepPin.write(0)
        time.sleep(0.001)

    time.sleep(0.4)

In [ ]:
def _init():
    _resetMotor()

    for octave in range(4):
        for note in range(12):
            floppyDelay[octave][note] = floppyConv / freq[octave][note]

In [ ]:
import time
def millis():
    return round(time.time() * 1000)

def delayMicroseconds(seconds):
    time.sleep(seconds/1000000.0)

In [ ]:
def _playNote(note, octave, length):
    if note == Zz:
        time.sleep(length/1000)
        return
    
    _dir = 1
    pause = floppyDelay[octave][note] * 10

    endTime = millis() + length
    while (millis() < endTime):
    #while(noteStatus):
        dirPin.write(_dir)
        if _dir == 0:
            _dir = 1
        else:
            _dir = 0

        stepPin.write(1)
        stepPin.write(0)
        delayMicroseconds(pause)

In [ ]:
def _rest(length):
    endTime = time.time() + length/1000
    while time.time() < endTime:
        time.sleep(0.005)

In [ ]:
_init()

In [ ]:
_playNote(Cs, 3, 1000)

In [ ]:
# super mario: EE E CE G -G

_playNote(Es, 3, 200)
_playNote(Zz, 1, 50)
_playNote(Es, 3, 200)
_playNote(Zz, 1, 150)
_playNote(Es, 3, 200)
_playNote(Zz, 1, 200)
_playNote(Cs, 3, 200)
_playNote(Zz, 1, 80)
_playNote(Es, 3, 200)
_playNote(Zz, 1, 200)
_playNote(Gs, 3, 150)



# Microblaze mode

In [3]:
base = BaseOverlay("base.bit")

In [4]:
%%microblaze base.ARDUINO
#include <pyprintf.h>
#include "gpio.h"
#include <unistd.h>
#include <time.h>
#include <timer.h>

gpio dirPin;
gpio stepPin;

// Define an octave with naturals and sharps (Zz = rest)
enum { Cn, Cs, Dn, Ds, En, Fn, Fs, Gn, Gs, An, As, Bn, Zz };

// Define another one with flats and remaining sharps
enum { Bs, Df, Dn2, Ef, En2, Es, Gf, Gn2, Af, An2, Bf, Bn2, Zz2 };

/**
 * Frequencies in hundredths of Hz, e.g. middle A = 44000
 * 4 Octaves with 12 notes per octave, i.e. C to B
 */
const int freq[4][12] = {
    { 13081,13859,14683,15556,16481,17461,18500,19600,20765,22000,23308,24694 },
    { 26163,27718,29366,31113,32963,34923,36999,39200,41530,44000,46616,49388 },
    { 52325,55437,58733,62225,65925,69846,73999,78399,83061,88000,93233,98777 },
    { 104650,110873,117466,124451,131851,139691,147998,156798,166122,176000,186466,197553 }
};
/**
 * Frequency (in Hz) is converted to Floppy Delay using the formula:
 *   314000 / frequency = floppy delay
 * so middle A = 314000 / 440 = 714
 *
 * Lowest realistic note is delay = 1550
 * Highest realistic note is delay = 210
 */
const int floppyConv = 31400000;
// Calculate all our floppy delays at the start
int floppyDelay[4][12];
// Song1 is the C major scale (note, octave, length)
const int song1_tempo = 120;

int noteStatus = 0;
int actual_note;
int actual_octave;

#define HIGH 1
#define LOW 0

void delay(int milliseconds) {
    delay_ms(milliseconds);
}

void delayMicroseconds(int microseconds) {
    usleep(microseconds);
}


void digitalWrite(gpio gp, int value){
    gpio_write(gp, value);
}


void gpio_config(){
    dirPin = gpio_open(8);
    gpio_set_direction(dirPin, GPIO_OUT);

    stepPin = gpio_open(9);
    gpio_set_direction(stepPin, GPIO_OUT);

}

void _resetMotor()
{
    // To reset head position move back 10 then forward 5
    digitalWrite(dirPin, LOW);
    for (int i=0; i < 10; i++){
        digitalWrite(stepPin, HIGH);
        digitalWrite(stepPin, LOW);
        delay(1);
    }

    digitalWrite(dirPin, HIGH);
    for (int i=0; i < 5; i++){
        digitalWrite(stepPin, HIGH);
        digitalWrite(stepPin, LOW);
        delay(1);
    }

    delay(400);
}

int _init()
{

    gpio_config();

    _resetMotor();

    for (int octave = 0; octave < 4; octave++){
        for (int note = 0; note < 12; note++){
            floppyDelay[octave][note] = floppyConv / freq[octave][note];
        }
    }

    pyprintf("Ready!");
    return 0;
}

void _playNote(int note, int octave, int length)
{
    static int dir = 1;
    int pause = floppyDelay[octave][note] * 10;

    int endTime = length;
    
    if(note == Zz){
        delayMicroseconds(length);
        return;
    }
    
    while (endTime > 0){
        digitalWrite(dirPin, dir);
        if (dir == 0)
            dir = 1;
        else
            dir = 0;

        digitalWrite(stepPin, HIGH);
        digitalWrite(stepPin, LOW);
        delayMicroseconds(pause);
        endTime -= 1;
    }
    digitalWrite(stepPin, LOW);
    digitalWrite(dirPin, LOW);    
}

int _rest(int length)
{
    int endTime = length;
    digitalWrite(stepPin, LOW);
    digitalWrite(dirPin, LOW);

    while (endTime > 0){
        //pyprintf("Ciao %d\n", endTime);
        delay(1);
        endTime -= 1;
    }
    return endTime;
}



In [5]:
_init()

Ready!

0

In [8]:
# super mario: EE E CE G -G

_playNote(Es, 2, 25)
_rest(80)
_playNote(Es, 2, 20)
_rest(200)
_playNote(Es, 2, 25)
_rest(200)
_playNote(Cs, 2, 20)
_rest(80)
_playNote(Es, 2, 25)
_rest(250)
_playNote(Gs, 2, 25)
_rest(600)
_playNote(Gs, 1, 25)